<a href="https://colab.research.google.com/github/tthuy123/graduation/blob/main/CNN%2BLSTM_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

so this one code is re-implementation/adopted from this paper, i techinacally first follow their method to see if the result works. After that i will see hehehehehe


In [ ]:
import pandas as pd

In [ ]:
df_info = pd.read_csv("/content/drive/MyDrive/OULAD/studentInfo.csv")
df_vle = pd.read_csv("/content/drive/MyDrive/OULAD/studentVle.csv")
df_assess = pd.read_csv("/content/drive/MyDrive/OULAD/studentAssessment.csv")
df_assessments = pd.read_csv("/content/drive/MyDrive/OULAD/assessments.csv")
df_courses = pd.read_csv("/content/drive/MyDrive/OULAD/courses.csv")
df_regis = pd.read_csv("/content/drive/MyDrive/OULAD/studentRegistration.csv")

In [ ]:
df_assessments

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0
...,...,...,...,...,...,...
201,GGG,2014J,37443,CMA,229.0,0.0
202,GGG,2014J,37435,TMA,61.0,0.0
203,GGG,2014J,37436,TMA,124.0,0.0
204,GGG,2014J,37437,TMA,173.0,0.0


In [ ]:
display(df_assessments['assessment_type'].unique())

array(['TMA', 'Exam', 'CMA'], dtype=object)

In [ ]:
df_assess

In [ ]:
df_vle

,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1
...,...,...,...,...,...,...
10655275,GGG,2014J,675811,896943,269,3
10655276,GGG,2014J,675578,896943,269,1
10655277,GGG,2014J,654064,896943,269,3
10655278,GGG,2014J,654064,896939,269,1


In [ ]:
df_courses

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240
5,BBB,2014B,234
6,CCC,2014J,269
7,CCC,2014B,241
8,DDD,2013J,261
9,DDD,2014J,262


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE

In [ ]:
## Merge data
vle_agg = df_vle.groupby(['id_student', 'code_module', 'code_presentation'])['sum_click'].sum().reset_index()
vle_agg.rename(columns={'sum_click': 'total_clicks'}, inplace=True)
vle_agg


,id_student,code_module,code_presentation,total_clicks
0,6516,AAA,2014J,2791
1,8462,DDD,2013J,646
2,8462,DDD,2014J,10
3,11391,AAA,2013J,934
4,23629,BBB,2013B,161
...,...,...,...,...
29223,2698257,AAA,2013J,758
29224,2698535,CCC,2014B,786
29225,2698535,EEE,2013J,3455
29226,2698577,BBB,2014J,717


In [ ]:
df_info

In [ ]:
# 1. LOAD DỮ LIỆU & LÀM SẠCH CƠ BẢN
df_info['target'] = df_info['final_result'].apply(lambda x: 1 if x in ['Distinction', 'Pass'] else 0)
## df_info

In [ ]:
# 2. XỬ LÝ DỮ LIỆU TIME-SERIES (VLE CLICKS)
# goal: tạo chuỗi lịch sử tương tác của học sinh theo các tuần
MAX_WEEKS = 40 # Chọn 40 vì dựa theo bảng courses max length là 269 ngày xấp xỉ 38.4 tuần
def map_date_to_week(date_val):
  if date_val < 0:
    return 0
  w = date_val // 7

  return w if w < MAX_WEEKS else MAX_WEEKS -1
df_vle['week_idx'] = df_vle['date'].apply(map_date_to_week)

week_activity = df_vle.groupby(['id_student', 'code_module', 'code_presentation', 'week_idx'])['sum_click'].sum().reset_index()

pivot_data = week_activity.pivot_table(
    index=['id_student', 'code_module', 'code_presentation'],
    columns='week_idx',
    values='sum_click',
    fill_value=0
)
expected_cols = list(range(MAX_WEEKS)) # [0, 1, ..., 39]
pivot_data = pivot_data.reindex(columns=expected_cols, fill_value=0)

pivot_data.columns = [f'week_{i}' for i in pivot_data.columns] # Đổi tên cho rõ ràng
pivot_data.reset_index(inplace=True)
pivot_data


In [ ]:
# 3. XỬ LÝ DỮ LIỆU ASSESSMENT (ĐIỂM SỐ)
assess_full = pd.merge(df_assess, df_assessments, on='id_assessment', how='inner')



,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0
...,...,...,...,...,...,...,...,...,...,...
173907,37443,527538,227,0,60.0,GGG,2014J,CMA,229.0,0.0
173908,37443,534672,229,0,100.0,GGG,2014J,CMA,229.0,0.0
173909,37443,546286,215,0,80.0,GGG,2014J,CMA,229.0,0.0
173910,37443,546724,230,0,100.0,GGG,2014J,CMA,229.0,0.0


In [ ]:
## another version of gemini (it's crazy you know)
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE

# ==========================================
# 1. LOAD DỮ LIỆU & LÀM SẠCH CƠ BẢN
# ==========================================
# Giả định các df đã được load như trong hình bạn gửi
# df_info, df_vle, df_assess, df_assessments, df_courses, df_regis

# Loại bỏ các bản ghi bị rút học (Withdrawn) nếu muốn tập trung vào Pass/Fail (tùy bài toán)
# Nhưng theo tài liệu, ta giữ lại và coi Fail/Withdrawn là lớp 0 (Weak).
# Encode Target: Distinction/Pass -> 1, Fail/Withdrawn -> 0
df_info['target'] = df_info['final_result'].apply(lambda x: 1 if x in ['Distinction', 'Pass'] else 0)

# ==========================================
# 2. XỬ LÝ DỮ LIỆU TIME-SERIES (VLE CLICKS)
# ==========================================
# Mục tiêu: Tạo chuỗi lịch sử click cho mỗi sinh viên qua các tuần (Ví dụ: 39 tuần)

# Xử lý ngày âm (trước khi bắt đầu khóa) về tuần 0
df_vle['week'] = df_vle['date'].apply(lambda x: 0 if x < 0 else x // 7)
# Giới hạn số tuần (ví dụ OULAD thường xét đến tuần 39)
df_vle = df_vle[df_vle['week'] <= 39]

# Pivot table: Hàng là SV, Cột là Tuần (week_0, week_1, ...), Giá trị là tổng click
vle_pivot = df_vle.pivot_table(
    index=['id_student', 'code_module', 'code_presentation'],
    columns='week',
    values='sum_click',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Đổi tên cột tuần
week_cols = [c for c in vle_pivot.columns if isinstance(c, int)]
vle_pivot.rename(columns={c: f'week_{c}' for c in week_cols}, inplace=True)
ts_features = [f'week_{i}' for i in week_cols] # Danh sách tên cột time-series

print(f"Số lượng bước thời gian (Time-steps): {len(ts_features)}")

# ==========================================
# 3. XỬ LÝ DỮ LIỆU ASSESSMENT (ĐIỂM SỐ)
# ==========================================
# Gộp bảng điểm chi tiết với thông tin bài kiểm tra
assess_full = pd.merge(df_assess, df_assessments, on='id_assessment', how='inner')

# Tính điểm trung bình (weighted score nếu cần) của mỗi SV trong mỗi khóa
# Lưu ý: Điểm số cũng có thể coi là time-series, nhưng ở đây ta lấy trung bình làm đặc trưng tĩnh (đơn giản hóa)
score_agg = assess_full.groupby(['id_student', 'code_module', 'code_presentation'])['score'].mean().reset_index()
score_agg.rename(columns={'score': 'mean_score'}, inplace=True)
# Fill NaN điểm số bằng 0 (sinh viên không nộp bài)
score_agg['mean_score'].fillna(0, inplace=True)

# ==========================================
# 4. GỘP TẤT CẢ DỮ LIỆU (MERGE)
# ==========================================
# Bắt đầu từ bảng gốc studentInfo
df_final = df_info.copy()

# Merge với dữ liệu VLE (Time-series)
df_final = pd.merge(df_final, vle_pivot, on=['id_student', 'code_module', 'code_presentation'], how='left')

# Merge với dữ liệu điểm số
df_final = pd.merge(df_final, score_agg, on=['id_student', 'code_module', 'code_presentation'], how='left')

# Điền 0 cho những sinh viên không có dữ liệu VLE hoặc điểm (những người bỏ học sớm/không tương tác)
df_final[ts_features] = df_final[ts_features].fillna(0)
df_final['mean_score'] = df_final['mean_score'].fillna(0)

# ==========================================
# 5. XỬ LÝ STATIC FEATURES (ENCODING)
# ==========================================
# Chọn các cột nhân khẩu học
static_cols = ['gender', 'region', 'highest_education', 'imd_band', 'age_band', 'disability', 'num_of_prev_attempts', 'mean_score']
# One-Hot Encoding cho các biến categorical
df_final = pd.get_dummies(df_final, columns=['gender', 'region', 'highest_education', 'imd_band', 'age_band', 'disability'], drop_first=True)

# Cập nhật danh sách static features sau khi one-hot
final_static_cols = [c for c in df_final.columns if c not in ts_features + ['id_student', 'code_module', 'code_presentation', 'final_result', 'target', 'date_registration', 'date_unregistration']]

print(f"Số lượng đặc trưng tĩnh (Static features): {len(final_static_cols)}")

# ==========================================
# 6. FEATURE SELECTION (RFE) & SPLIT
# ==========================================
# Tách X và y
# Lưu ý: RFE thường chạy trên dữ liệu 2D phẳng. Ta sẽ chọn đặc trưng tĩnh quan trọng nhất.
X_static = df_final[final_static_cols]
X_ts = df_final[ts_features] # Time series clicks
y = df_final['target']

# Chuẩn hóa (Normalization) - Theo mục 3.2
scaler_static = MinMaxScaler()
X_static_scaled = pd.DataFrame(scaler_static.fit_transform(X_static), columns=X_static.columns)

scaler_ts = MinMaxScaler()
X_ts_scaled = pd.DataFrame(scaler_ts.fit_transform(X_ts), columns=X_ts.columns)

# Chia tập Train/Test TRƯỚC KHI SMOTE để tránh data leakage
X_train_static, X_test_static, X_train_ts, X_test_ts, y_train, y_test = train_test_split(
    X_static_scaled, X_ts_scaled, y, test_size=0.2, random_state=42
)

# Áp dụng RFE cho phần Static Data (để loại bỏ bớt các biến nhân khẩu học nhiễu)
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=10) # Chọn top 10 đặc trưng tĩnh
rfe.fit(X_train_static, y_train)
selected_static = X_train_static.columns[rfe.support_]
print("Top static features selected:", selected_static)

# Cập nhật lại tập dữ liệu chỉ với feature đã chọn
X_train_static = X_train_static[selected_static]
X_test_static = X_test_static[selected_static]

# ==========================================
# 7. SMOTE (XỬ LÝ MẤT CÂN BẰNG)
# ==========================================
# SMOTE cần đầu vào là 2D. Ta cần gộp static và time-series lại để sinh mẫu giả lập đồng bộ
X_train_full = pd.concat([X_train_static, X_train_ts], axis=1)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_full, y_train)

print("Shape sau SMOTE:", X_train_resampled.shape)

# Tách lại ra Static và Time-series sau khi SMOTE
X_train_static_res = X_train_resampled[selected_static]
X_train_ts_res = X_train_resampled[ts_features]

# ==========================================
# 8. RESHAPE DATA CHO CNN-LSTM (QUAN TRỌNG)
# ==========================================
# Để CNN-LSTM học được cả ngữ cảnh tĩnh và động, cách phổ biến là:
# "Repeat Vector": Lặp lại thông tin tĩnh cho mỗi bước thời gian và ghép vào chuỗi click.
# Input shape cuối cùng: (Samples, TimeSteps, Features)
# Features = 1 (click) + N (static features)

def create_3d_input(static_df, ts_df):
    samples = static_df.shape[0]
    timesteps = ts_df.shape[1]

    # Chuyển time-series thành (Samples, TimeSteps, 1)
    ts_array = ts_df.values.reshape(samples, timesteps, 1)

    # Chuyển static thành (Samples, 1, n_static) rồi lặp lại cho các timesteps
    static_array = static_df.values.reshape(samples, 1, -1)
    static_tiled = np.tile(static_array, (1, timesteps, 1)) # (Samples, TimeSteps, n_static)

    # Ghép lại: Tại mỗi tuần, ta có [số click, giới tính, vùng miền, điểm số...]
    return np.concatenate([ts_array, static_tiled], axis=2)

X_train_final = create_3d_input(X_train_static_res, X_train_ts_res)
# X_test không SMOTE, dùng dữ liệu gốc
X_test_final = create_3d_input(X_test_static, X_test_ts)

print("="*30)
print(f"Dữ liệu hoàn thiện sẵn sàng cho mô hình!")
print(f"Train shape: {X_train_final.shape} (Samples, TimeSteps, Features)")
print(f"Test shape:  {X_test_final.shape}")
print(f"Labels shape: {y_train_resampled.shape}")